<a href="https://colab.research.google.com/github/chiglo-k/Mindbox_Test_Task/blob/main/Spark_Test_wrk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Создание Spark Session
spark = SparkSession.builder \
    .appName("ProductCategoty") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark версия: {spark.version}")

Spark версия: 3.5.1


In [6]:
def create_test_dataframes():
    """Создание фреймов"""

    # Продукты (Индекс - Наименование продукции)
    products_data = [
        (1, "iPhone 15 Pro"),
        (2, "MacBook Air M2"),
        (3, "Magic Mouse"),
        (4, "iPad Pro"),
        (5, "AirPods Pro"),
        (6, "Товар без категории"),
        (7, "Apple Watch"),
        (8, "Неизвестный Продукт")
    ]

    products_df = spark.createDataFrame(products_data, ["product_id", "product_name"])

    # Категории (Индекс - Категория продукции)
    categories_data = [
        (1, "Смартфоны"),
        (2, "Ноутбуки"),
        (3, "Аксессуары"),
        (4, "Планшеты"),
        (5, "Аудио"),
        (6, "Носимые устройства")
    ]

    categories_df = spark.createDataFrame(categories_data, ["category_id", "category_name"])

    # Связи многие-ко-многим
    relations_data = [
        (1, 1),  # iPhone -> Смартфоны
        (2, 2),  # MacBook -> Ноутбуки
        (3, 3),  # Mouse -> Аксессуары
        (4, 4),  # iPad -> Планшеты
        (4, 3),  # iPad -> Аксессуары
        (5, 5),  # AirPods -> Аудио
        (5, 3),  # AirPods -> Аксессуары
        (7, 6),  # Apple Watch -> Носимые устройства
        (7, 3),  # Apple Watch -> Аксессуары
        # Продукты 6 и 8 без категорий
    ]

    relations_df = spark.createDataFrame(relations_data, ["product_id", "category_id"])

    return products_df, categories_df, relations_df

products_df, categories_df, relations_df = create_test_dataframes()

print("Исходные данные созданы:")
print("Продукты:")
products_df.show()
print("Категории:")
categories_df.show()
print("Связи продукт-категория:")
relations_df.show()

Исходные данные созданы:
Продукты:
+----------+-------------------+
|product_id|       product_name|
+----------+-------------------+
|         1|      iPhone 15 Pro|
|         2|     MacBook Air M2|
|         3|        Magic Mouse|
|         4|           iPad Pro|
|         5|        AirPods Pro|
|         6|Товар без категории|
|         7|        Apple Watch|
|         8|Неизвестный Продукт|
+----------+-------------------+

Категории:
+-----------+------------------+
|category_id|     category_name|
+-----------+------------------+
|          1|         Смартфоны|
|          2|          Ноутбуки|
|          3|        Аксессуары|
|          4|          Планшеты|
|          5|             Аудио|
|          6|Носимые устройства|
+-----------+------------------+

Связи продукт-категория:
+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          2|
|         3|          3|
|         4|          4|
|         4|          3|
|

In [17]:
def get_all_product_category_pairs(products_df, categories_df, relations_df):
    """
    Создание единого фрейма
    """

    # LEFT JOIN: продукты -> связи -> категории
    result = products_df \
        .join(relations_df, on="product_id", how="left") \
        .join(categories_df, on="category_id", how="left") \
        .select("product_name", "category_name") \
        .orderBy("product_name", "category_name")

    return result

result_df = get_all_product_category_pairs(products_df, categories_df, relations_df)


print("\nВсе пары продукт-категория:")
result_df.show(truncate=False)
result_df_wth_category = result_df.where('category_name IS NULL')
print("\nПродукты без категории")
result_df_wth_category.show()


Все пары продукт-категория:
+-------------------+------------------+
|product_name       |category_name     |
+-------------------+------------------+
|AirPods Pro        |Аксессуары        |
|AirPods Pro        |Аудио             |
|Apple Watch        |Аксессуары        |
|Apple Watch        |Носимые устройства|
|MacBook Air M2     |Ноутбуки          |
|Magic Mouse        |Аксессуары        |
|iPad Pro           |Аксессуары        |
|iPad Pro           |Планшеты          |
|iPhone 15 Pro      |Смартфоны         |
|Неизвестный Продукт|NULL              |
|Товар без категории|NULL              |
+-------------------+------------------+


Продукты без категории
+-------------------+-------------+
|       product_name|category_name|
+-------------------+-------------+
|Неизвестный Продукт|         NULL|
|Товар без категории|         NULL|
+-------------------+-------------+

